In [13]:
from pyspark.sql.functions import when, col, udf, expr
from pyspark.sql.types import StringType

import reverse_geocoder as rg

StatementMeta(, 0d554109-9f66-4a98-aae3-a832d1342997, 17, Finished, Available, Finished)

In [14]:
## Retrieve data from silver layer
df = spark.read.table("earthquake_events_silver").filter(col('time') > start_date)

StatementMeta(, 0d554109-9f66-4a98-aae3-a832d1342997, 18, Finished, Available, Finished)

In [15]:
## Begin further cleaning by adding specific attributes which are useful for our Power BI report
def get_country_code(lat, lon):

    coordinates = (float(lat), float(lon))

    # country code for use in BI logic
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, 0d554109-9f66-4a98-aae3-a832d1342997, 19, Finished, Available, Finished)

In [16]:
# Registered UDF to apply on Spark DataFrames
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, 0d554109-9f66-4a98-aae3-a832d1342997, 20, Finished, Available, Finished)

In [17]:
## Apply UDF
df_with_location = df.withColumn('country_code', get_country_code_udf(col('latitude'), col('longitude')))

StatementMeta(, 0d554109-9f66-4a98-aae3-a832d1342997, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d92b8153-3273-46f5-97f4-8496c0838bf4)

In [18]:
## Add significance classifier
df_with_location_sig_class = (df_with_location
                            .withColumn('sig_class', 
                            expr('CASE \
                            WHEN sig < 100 THEN "Low" \
                            WHEN (sig >= 100) AND (sig < 500) THEN "Moderate" \
                            ELSE "High" END')
))

StatementMeta(, 0d554109-9f66-4a98-aae3-a832d1342997, 22, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1836340f-352a-4952-a7ce-f5665b829aab)

In [19]:
df_with_location_sig_class.write.mode("append").saveAsTable('earthquake_events_gold')

StatementMeta(, 0d554109-9f66-4a98-aae3-a832d1342997, 23, Finished, Available, Finished)